In [ ]:
!nvidia-smi

Sat Nov  6 13:36:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    39W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !mkdir /content/drive/MyDrive/EVA6/Capstone/dataset

In [ ]:
# !unzip /content/drive/MyDrive/EVA6/Capstone/construction_materials_dataset.zip -d /content/drive/MyDrive/EVA6/Capstone/dataset/

In [ ]:
! git clone https://github.com/gokul-pv/EVA6_Capstone

Cloning into 'EVA6_Capstone'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 136 (delta 30), reused 86 (delta 15), pack-reused 0
Receiving objects: 100% (136/136), 16.10 MiB | 25.56 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
! pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-vr96cms3
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-vr96cms3
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=4821143a2e61313b16bba79816c87411334f5fa30e4fcfa714c0174e85fb395e
  Stored in directory: /tmp/pip-ephem-wheel-cache-9x_giuux/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [ ]:
import os
import sys
import numpy as np
import torch
from torch import nn
torch.set_grad_enabled(False);
print(torch.__version__)

1.9.0+cu111


In [ ]:
sys.path.append(os.path.join(os.getcwd(), "/content/EVA6_Capstone/"))

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
use_cuda, device

(True, device(type='cuda'))

In [ ]:
import torchvision.transforms as T

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Load detr model
model, postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet101_panoptic', pretrained=True, return_postprocessor=True, num_classes=250)
# Convert to eval mode
model = model.to(device)
model.eval()

print("Model Loaded")

Using cache found in /root/.cache/torch/hub/facebookresearch_detr_master


Model Loaded


In [ ]:
def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

In [ ]:
from DatasetCreation import COCO_CATEGORIES, CUSTOM_CATEGORIES, COCO_NAMES, MAPPINGS, INFO, LICENSES, cat2id

In [ ]:
from DatasetCreation import get_original_mask, get_annotation_info, create_image_info

In [ ]:
import datetime
import time
import json
import traceback
from pathlib import Path

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import requests
import panopticapi
from panopticapi.utils import id2rgb, rgb2id, IdGenerator
import io
import matplotlib.pyplot as plt

In [ ]:
id2catdetail = {category['id']: category for category in CUSTOM_CATEGORIES}

id_generator = IdGenerator(id2catdetail)

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/EVA6/Capstone/dataset'

processing_file = ""
processing_data = []

image_id = 1
annotation_id = 1
segment_id = 1

GLOBAL_COCO = {
    "licenses": LICENSES,
    "info": INFO,
    "categories": CUSTOM_CATEGORIES,
    "annotations": [],
    "images": []
}

GLOBAL_PANOPTIC = {
    "licenses": LICENSES,
    "info": INFO,
    "categories": CUSTOM_CATEGORIES,
    "annotations": [],
    "images": []
}

In [ ]:
data_path = [
     '/content/drive/MyDrive/EVA6/Capstone/dataset/aac_blocks',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/adhesives',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/ahus',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/aluminium_frames_for_false_ceiling',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/chiller',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/concrete_mixer_machine',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/concrete_pump',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/control_panel',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/cu_piping',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/distribution_transformer',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/dump_truck_tipper_truck',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/emulsion_paint',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/enamel_paint',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/fine_aggregate',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/fire_buckets',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/fire_extinguishers',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/glass_wool',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/grader',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/hoist',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/hollow_concrete_blocks',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/hot_mix_plant',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/hydra_crane',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/interlocked_switched_socket',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/junction_box',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/lime',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/marble',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/metal_primer',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/pipe_fittings',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/rcc_hume_pipes',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/refrigerant_gas',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/river_sand',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/rmc_batching_plant',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/rmu_units',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/sanitary_fixtures',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/skid_steer_loader',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/smoke_detectors',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/split_units',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/structural_steel_channel',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/switch_boards_and_switches',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/texture_paint',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/threaded_rod',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/transit_mixer',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/vcb_panel',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/vitrified_tiles',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/vrf_units',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/water_tank',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/wheel_loader',
     '/content/drive/MyDrive/EVA6/Capstone/dataset/wood_primer'  	  	  	  
]

In [ ]:
for category_path in data_path:

    # store starting time
    start = time.time()

    # get category name
    category_name = category_path.split("/")[7]
    print("Processing Category:", category_name)

    # open category coco file
    with open(os.path.join(category_path, "coco.json"), "r") as coco_file:
        category_coco = json.load(coco_file)
        
    images_root = os.path.join(category_path, 'images')
    
    TEMP_COCO_IMAGES = {}
    
    # Run over all images
    for im in category_coco["images"]:
        im['annotations'] = []
        TEMP_COCO_IMAGES[im['id']] = im

   
    for ann in category_coco["annotations"]:
        TEMP_COCO_IMAGES[ann['image_id']]["annotations"].append(ann)
      
    for i, image_coco in TEMP_COCO_IMAGES.items():
        # get image path
        ## This data can be used further for logging if failed while processing
        processing_file = os.path.join(images_root, image_coco['file_name'])
        processing_data = image_coco
        output_file_name = category_name + "_" + str(image_id) + ".jpg"
        output_file_path = os.path.join(ROOT_DIR, "images", output_file_name)
        
        output_mask_name = category_name + "_" + str(image_id) + ".png"
        output_mask_path = os.path.join(ROOT_DIR, "masks", output_mask_name)

        try:

          # Read this image and get shape of image
          imo = Image.open(processing_file).convert('RGB')

          try:
              h, w, c = np.array(imo).shape
          except:
              h, w = np.array(imo).shape
              c = 1

          # if no of channels != 3, open the image and convert it to 3 channel - RGB
          if c == 4 or c == 1:
              imo = imo.convert('RGB')
              h, w, c = np.array(imo).shape

          # Create a copy of image this will be used for further processing
          im = imo.copy()

          # Apply transform and convert image to batch
          # mean-std normalize the input image (batch-size: 1)
          img = transform(im).unsqueeze(0).to(device)  # [h, w, c] -> [1, c, ht, wt]

          # Generate output for image
          out = model(img)

          # Generate score
          # compute the scores, excluding the "no-object" class (the last one)
          scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]

          # threshold the confidence
          keep = scores > 0.85

          # Keep only ones above threshold
          pred_logits, pred_boxes = out["pred_logits"][keep][:, :len(COCO_NAMES) - 1], out["pred_boxes"][keep]    

          # the post-processor expects as input the target size of the predictions (which we set here to the image size)
          result = postprocessor(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]    

          # The segmentation is stored in a special-format png
          panoptic_seg = Image.open(io.BytesIO(result['png_string'])).resize((w, h), Image.NEAREST)

           # (wp, hp) = panoptic_seg.size
          panoptic_seg = np.array(panoptic_seg, dtype=np.uint8).copy()   

          # We retrieve the ids corresponding to each mask
          panoptic_seg_id = rgb2id(panoptic_seg)

          # Merge predicted annotations      
          # 1. Get Mapping from predicted category id to new category ids
          unique_category_id = []
          for i, segment in enumerate(result['segments_info']):
              result['segments_info'][i]["category_id"] = MAPPINGS[result['segments_info'][i]["category_id"]]
              if result['segments_info'][i]["category_id"] not in unique_category_id:
                  unique_category_id.append(result['segments_info'][i]["category_id"])

          # Sort array
          unique_category_id.sort()

          unique_category_id_to_id =  {category_id: i for i, category_id in enumerate(unique_category_id)}
          unique_id_to_category_id =  {i: category_id for category_id, i in unique_category_id_to_id.items()}   

          for i, segment in enumerate(result['segments_info']):
              result['segments_info'][i]["new_id"] = unique_category_id_to_id[result['segments_info'][i]["category_id"]]

          # Update original panoptic_seg_id array with new ids as the new segmentation combines different categories.
          custom_panoptic_seg_id = np.zeros((panoptic_seg_id.shape[0], panoptic_seg_id.shape[1]), dtype=np.uint8)
          
          # Update this custom panoptic seg matrix
          for i, segment in enumerate(result['segments_info']):
              custom_panoptic_seg_id[result['segments_info'][i]['id'] == panoptic_seg_id] = result['segments_info'][i]['new_id']

          # Create new Segmentation info         
          custom_panoptic_segments_info = []
          for category_id in unique_category_id:
              custom_panoptic_segments_info.append({
                  'segment_id': unique_category_id_to_id[category_id], 
                  'category_id': category_id,
                  'bbox': [],
                  'area': 0,
                  'iscrowd': 0,
                  'isthing': 0
              })

          # annotations of our construction things
          omask = processing_data['annotations']
          
          TEMP_ANNOTATIONS = []

          # Overlay things mask one at a time
          for annotation in omask:
              # overlay mask of construction things on top of detr output
              omask_image_id = get_original_mask((h, w), annotation)
              custom_panoptic_seg_id[omask_image_id.astype(np.bool_)] = custom_panoptic_seg_id.max() + 1
              custom_panoptic_segments_info.append({
                  'segment_id': custom_panoptic_seg_id.max(), 
                  'category_id': cat2id[category_name], 
                  'bbox': annotation['bbox'],
                  'area': annotation['area'],
                  'iscrowd': 0,
                  'isthing': 1
              })

              # append annotation of construction things in json file
              annotation["category_id"] = cat2id[category_name]
              annotation["image_id"] = image_id
              annotation["id"] = custom_panoptic_seg_id.max()
              TEMP_ANNOTATIONS.append(annotation)

          # Convert to binary segment
          binary_masks = np.zeros((
              custom_panoptic_seg_id.max() + 1,
              custom_panoptic_seg_id.shape[0],
              custom_panoptic_seg_id.shape[1]),
              dtype=np.uint8
          )

          # for each binary mask, detect contours and create annotation for those contours
          if len(unique_category_id):
              # Skip the ones which are added by us
              for category_id in unique_category_id:
                  binary_masks[unique_category_id_to_id[category_id], :, :] = custom_panoptic_seg_id == unique_category_id_to_id[category_id]
                  annotation_info = get_annotation_info(binary_masks[unique_category_id_to_id[category_id]], None, image_id, category_id, unique_category_id_to_id[category_id], 0)
                  
                  if annotation_info is not None:
                      TEMP_ANNOTATIONS.append(annotation_info) 
                      custom_panoptic_segments_info[unique_category_id_to_id[category_id]]['bbox'] = annotation_info['bbox']
                      custom_panoptic_segments_info[unique_category_id_to_id[category_id]]['area'] = annotation_info['area']   
          else:
              # Do something for the once where there are no predictions
              ## Probably mark them as None
              pass

          # Write data to global json and save files to image dir's
          # save image to new path as .jpg
          imo.save(output_file_path)

          # save panoptic image
          Image.fromarray(id2rgb(custom_panoptic_seg_id), 'RGB').save(output_mask_path)

          # Create a new empty id array
          panoptic_seg_id_corrected = np.zeros(custom_panoptic_seg_id.shape)

          # All unique segments in original id image
          unique_panoptic_seg_id = np.unique(custom_panoptic_seg_id)
          
          # All segments for which masks are available
          segms_corrected = []

          # Finally we get id for each mask individually
          for segm in custom_panoptic_segments_info:

              # Original Segment id
              segment_id = segm["segment_id"]
              
              if segment_id in unique_panoptic_seg_id:
                  # Get new Segment Id
                  proper_id = id_generator.get_id(segm['category_id'])
                  # Populate these new segments id
                  panoptic_seg_id_corrected[custom_panoptic_seg_id==segm["segment_id"]] = proper_id
                  # Update segm annotation json
                  segm['id'] = proper_id
                  # if segm id is less than max value for original mask print it
                  if proper_id < panoptic_seg_id.max()+1:
                      print(proper_id)
                      
                  # Append this Segment
                  segms_corrected.append(segm)
                  
          # Update original segments_info
          custom_panoptic_segments_info = segms_corrected
          
          # Save Image
          Image.fromarray(id2rgb(panoptic_seg_id_corrected), 'RGB').save(f"/content/drive/MyDrive/EVA6/Capstone/dataset/panoptic/{output_mask_name}")

          # create image_info object and append it to original list
          image_info = create_image_info(image_id, output_file_name, imo.size)
          image_info["original_file"] = processing_file

          # Only consider images where segment info is correctly available
          if len(segms_corrected):
            GLOBAL_COCO["images"].append(image_info)
            GLOBAL_PANOPTIC["images"].append(image_info)

            for annotation in TEMP_ANNOTATIONS:
                annotation["id"] = annotation_id
                GLOBAL_COCO["annotations"].append(annotation)
                annotation_id += 1
              
            for segment_info in custom_panoptic_segments_info:
                # segment_info["id"] = segment_id
                segment_id += 1

            GLOBAL_PANOPTIC["annotations"].append({
                "segments_info": custom_panoptic_segments_info,
                "file_name": output_mask_name,
                "image_id": image_id
            })

          # increment the image_count
          image_id += 1


        except Exception as e:
            # if there is any error, add info about it in errros file and procees to next image
            print("Error occurred while processig file:", processing_file)
            
            with open(os.path.join(ROOT_DIR, "error.json"), 'r') as error_file:
                error_json = json.load(error_file)
                
            with open(os.path.join(ROOT_DIR, "error.json"), 'w') as error_file:
                error_json["error"].append({
                    "processing_file": processing_file,
                    "processing_data": processing_data
                })
                
                json.dump(error_json, error_file)
                
            traceback.print_exc()

        del img, out, scores, keep, pred_logits, pred_boxes, result,panoptic_seg_id, panoptic_seg 
        torch.cuda.empty_cache() 
      
    total_time_str = str(datetime.timedelta(seconds=int(time.time() - start)))
    print(f"Completed Category: {category_name}, Time Taken: {total_time_str}")

    # open the final json, and commit changes in that file
    with open(os.path.join(ROOT_DIR, "coco.json"), 'w') as output_json_file:
        json.dump(GLOBAL_COCO, output_json_file)
        
    with open(os.path.join(ROOT_DIR, "panoptic.json"), 'w') as output_json_file:
        json.dump(GLOBAL_PANOPTIC, output_json_file, default=convert)
        
    print(image_id, annotation_id, segment_id)   

Processing Category: skid_steer_loader


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Completed Category: skid_steer_loader, Time Taken: 0:02:01
6590 23905 24023
Processing Category: smoke_detectors
Completed Category: smoke_detectors, Time Taken: 0:00:46
6640 23999 24117
Processing Category: split_units
Completed Category: split_units, Time Taken: 0:07:49
7178 25666 25796
Processing Category: structural_steel_channel


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Completed Category: structural_steel_channel, Time Taken: 0:04:09
7579 26563 26694
Processing Category: switch_boards_and_switches
Completed Category: switch_boards_and_switches, Time Taken: 0:05:17
8079 27599 27763
Processing Category: texture_paint
Completed Category: texture_paint, Time Taken: 0:00:31
8131 27702 27868
Processing Category: threaded_rod
Error occurred while processig file: /content/drive/MyDrive/EVA6/Capstone/dataset/threaded_rod/images/img_052.jpg


Traceback (most recent call last):
  File "<ipython-input-29-155ae7a1607d>", line 42, in <module>
    imo = Image.open(processing_file).convert('RGB')
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EVA6/Capstone/dataset/threaded_rod/images/img_052.jpg'


Error occurred while processig file: /content/drive/MyDrive/EVA6/Capstone/dataset/threaded_rod/images/img_050.jpg


Traceback (most recent call last):
  File "<ipython-input-29-155ae7a1607d>", line 42, in <module>
    imo = Image.open(processing_file).convert('RGB')
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EVA6/Capstone/dataset/threaded_rod/images/img_050.jpg'


Completed Category: threaded_rod, Time Taken: 0:21:32
8682 29041 29207
Processing Category: transit_mixer
Completed Category: transit_mixer, Time Taken: 0:00:40
8732 29290 29456
Processing Category: vcb_panel
Completed Category: vcb_panel, Time Taken: 0:06:51
9232 30979 31166
Processing Category: vitrified_tiles
Completed Category: vitrified_tiles, Time Taken: 0:04:21
9632 32510 32733
Processing Category: vrf_units
Completed Category: vrf_units, Time Taken: 0:00:37
9682 32632 32855
Processing Category: water_tank
Completed Category: water_tank, Time Taken: 0:03:31
9986 34003 34229
Processing Category: wheel_loader
Completed Category: wheel_loader, Time Taken: 0:02:06
10136 34697 34931
Processing Category: wood_primer
Completed Category: wood_primer, Time Taken: 0:00:09
10148 34726 34961


In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.3 GB
Cached:    0.9 GB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [ ]:
torch.cuda.empty_cache()